# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [80]:
# API
import requests
import json

# Data
import pandas as pd
import numpy as np

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas # maps

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [83]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head(25)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ormeau,-27.7833,153.2667,51.12,85.0,70.0,4.79,AU,1.717444e+09
1,1,ushuaia,-54.8000,-68.3000,40.66,70.0,40.0,5.75,AR,1.717444e+09
2,2,akureyri,65.6835,-18.0878,35.53,87.0,100.0,27.63,IS,1.717444e+09
3,3,berezovyy,51.6667,135.7000,42.26,93.0,60.0,1.16,RU,1.717444e+09
4,4,badger,64.8000,-147.5333,63.36,55.0,75.0,3.44,US,1.717444e+09
5,5,iqaluit,63.7506,-68.5145,33.53,86.0,100.0,15.01,CA,1.717444e+09
6,6,edinburgh of the seven seas,-37.0676,-12.3116,53.62,59.0,47.0,5.75,SH,1.717444e+09
7,7,adam,22.3756,57.5233,83.70,62.0,5.0,10.16,OM,1.717444e+09
8,8,west island,-12.1568,96.8225,82.38,89.0,8.0,12.66,CC,1.717444e+09
9,9,kerouane,9.2667,-9.0167,72.81,96.0,100.0,4.90,GN,1.717444e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [76]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [87]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 80.6) & (city_data_df["Max Temp"] > 69.8) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
191,191,nsanje,-16.9200,35.2620,71.85,61.0,0.0,0.69,MW,1.717444e+09
375,375,barcelos,41.5388,-8.6151,73.54,72.0,0.0,2.93,PT,1.717444e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [85]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
191,nsanje,MW,-16.9200,35.2620,61.0,
375,barcelos,PT,41.5388,-8.6151,72.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [86]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nsanje - nearest hotel: No hotel found
barcelos - nearest hotel: Hotel Dom Nuno


,City,Country,Lat,Lng,Humidity,Hotel Name
191,nsanje,MW,-16.9200,35.2620,61.0,No hotel found
375,barcelos,PT,41.5388,-8.6151,72.0,Hotel Dom Nuno


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot